<a href="https://colab.research.google.com/github/dhy02094/pytorch/blob/master/%EC%9E%91%EB%AC%BC_%EC%9E%8E_%EC%82%AC%EC%A7%84%EC%9C%BC%EB%A1%9C_%EC%A7%88%EB%B3%91_%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd "/content/drive/MyDrive/Colab Notebooks/pytorch/작물 잎 사진"


/content/drive/MyDrive/Colab Notebooks/pytorch/작물 잎 사진


In [7]:
!unzip -qq "/content/drive/MyDrive/Colab Notebooks/pytorch/작물 잎 사진/dataset.zip의 사본"

데이터 분할을 위한 폴더 생성

In [8]:
import os
import shutil

original_dataset_dir = '/content/drive/MyDrive/Colab Notebooks/pytorch/작물 잎 사진/dataset'                    # 저장된 데이터 공간
classes_list = os.listdir(original_dataset_dir)       # 저장된 데이터 공간에서 모든 폴더이름 가져오기

base_dir = '/content/drive/MyDrive/Colab Notebooks/pytorch/작물 잎 사진/splitted'                               # 나눈 데이터를 앞으로 저장할 폴더
os.mkdir(base_dir)                                    

train_dir = os.path.join(base_dir, 'train')           # 경로(base_dir/tarin)
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'val')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

for clss in classes_list :                           # 모든 폴더이름을 트레인, 검증, 테스트에 넣기
    os.mkdir(os.path.join(train_dir, clss))
    os.mkdir(os.path.join(validation_dir, clss))
    os.mkdir(os.path.join(test_dir, clss))



데이터 분할과 클래스별 데이터 수 확인

In [9]:
import math

for clss in classes_list :
    path = os.path.join(original_dataset_dir, clss)   #권한을 주는건데 original - clss 로 연결
    fnames = os.listdir(path)      # 경로를 넣으니까 fnames는 저 위를 다 갖는거

# 데이터 개수 분배
    train_size = math.floor(len(fnames) * 0.6)
    validation_size = math.floor(len(fnames) * 0.2)
    test_size = math.floor(len(fnames) * 0.2)

    train_fnames = fnames[:train_size]
    print('Train size(',clss,'): ', len(train_fnames))
    for fname in train_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(train_dir, clss), fname)
        shutil.copyfile(src, dst)

    validation_fnames = fnames[train_size:(validation_size + train_size)]
    print('Validation size(',clss,'): ', len(validation_fnames))
    for fname in validation_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(validation_dir, clss), fname)
        shutil.copyfile(src, dst)

    test_fnames = fnames[(train_size + validation_size): 
                                (validation_size + train_size + test_size)]
    
    print('Test size(',clss,'): ', len(test_fnames))
    for fname in test_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(test_dir, clss), fname)
        shutil.copyfile(src, dst)


    

    

Train size( Pepper,_bell___healthy ):  886
Validation size( Pepper,_bell___healthy ):  295
Test size( Pepper,_bell___healthy ):  295
Train size( Grape___Esca_(Black_Measles) ):  829
Validation size( Grape___Esca_(Black_Measles) ):  276
Test size( Grape___Esca_(Black_Measles) ):  276
Train size( Pepper,_bell___Bacterial_spot ):  598
Validation size( Pepper,_bell___Bacterial_spot ):  199
Test size( Pepper,_bell___Bacterial_spot ):  199
Train size( Strawberry___healthy ):  273
Validation size( Strawberry___healthy ):  91
Test size( Strawberry___healthy ):  91
Train size( Grape___Black_rot ):  708
Validation size( Grape___Black_rot ):  236
Test size( Grape___Black_rot ):  236
Train size( Corn___Common_rust ):  715
Validation size( Corn___Common_rust ):  238
Test size( Corn___Common_rust ):  238
Train size( Apple___Apple_scab ):  378
Validation size( Apple___Apple_scab ):  126
Test size( Apple___Apple_scab ):  126
Train size( Potato___healthy ):  91
Validation size( Potato___healthy ):  30


베이스라인 모델 학습을 위한 준비

In [10]:
import torch

USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')

print('Current cuda device is', DEVICE)

# 오 코랩은 cuda 로 돌리는게 가능하다!

Current cuda device is cuda


In [11]:
BATCH_SIZE = 256
EPOCH = 30

import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder   # 계층적인 폴더 구조를 가지고 있는 데이터셋을 불러올때 사용

transform_base = transforms.Compose([transforms.Resize((64,64)),
                                     transforms.ToTensor()])
# transforms.Compose (이미지 전처리 , Augmentation 에 사용)
# augmentation을 통해 좌우 반전, 발긱 조절, 이미지 확대가 가능. 여기선 64,64 로 조정하고 Totensor로 변환하면서 0~1 정규화

train_dataset = ImageFolder(root='/content/drive/MyDrive/Colab Notebooks/pytorch/작물 잎 사진/splitted/train',transform = transform_base)

val_dataset = ImageFolder(root='/content/drive/MyDrive/Colab Notebooks/pytorch/작물 잎 사진/splitted/val',
                          transform = transform_base)

from torch.utils.data import DataLoader

train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size = BATCH_SIZE,
                                           shuffle = True,
                                           num_workers=4)
val_loader = torch.utils.data.DataLoader(val_dataset,
                                         batch_size = BATCH_SIZE,
                                         shuffle = True,
                                         num_workers = 4) 
# num_workers 는 gpu 얼마나 할당할지.

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [12]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):
    # 속성 생성
    def __init__(self):      

        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(3, 32, 3, padding = 1)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(32,64,3, padding = 1)
        self.conv3 = nn.Conv2d(64,64,3, padding = 1)

        self.fc1 = nn.Linear(4096, 512)
        self.fc2 = nn.Linear(512, 33)

    def forward(self, x):

        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p = 0.25, training = self.training)

        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training= self.training)

        x = self.conv3(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p = 0.25, training = self.training)

        x = x.view(-1,4096)
        x = self.fc1(x)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training = self.training)
        x = self.fc2(x)

        return F.log_softmax(x,dim=1)

model_base = Net().to(DEVICE)
optimizer = optim.Adam(model_base.parameters(), lr=0.001)


[모델 학습을 위한 학습]

In [14]:
def train(model, train_loader, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):  #target이 뭔가 싶긴한데... 아마 폴더로 설정되나 보다.
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad() #이전 배치의 그래디언트값이 옵티마이저에 저장되어 있으므로 초기화
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

[모델 평가를 위한 학습]

In [15]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0  #미니 배치별로 Loss를 합산해서 저장할 변수
    correct = 0   #올바르게 예측한 데이터 수를 세는 변수

    with torch.no_grad():   # 모델 평가에서는 parameter를 업데이트 하지 않아야 한다. #이 메소드는 parameter업데이트를 중단
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)

            test_loss += F.cross_entropy(output,
                                         target, reduction = 'sum').item()
            
            pred = output.max(1, keepdim= True)[1] #33개의 클래스에 속할 각각의 확률값 출력

            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

                                         



[모델 학습 실행]

In [18]:
import time
import copy

def train_baseline(model, train_loader, val_loader,
                   optimizer, num_epochs =30):
    best_acc = 0 # 정확도 저장 변수
    best_model_wts = copy.deepcopy(model.state_dict())

    for epoch in range(1, num_epochs +1):
        since = time.time()
        train(model, train_loader, optimizer)
        train_loss, train_acc = evaluate(model, train_loader)
        val_loss, val_acc = evaluate(model, val_loader)

        if val_acc > best_acc:
            best_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())

        time_elapsed = time.time() - since # 한 epcoh마다 걸린 시간
        print('-----------------epoch {} ----------------------'.format(epoch))

        print('train Loss: {:.4f}, Accuracy: {:.2f}%'
                                    .format(train_loss, train_acc))
        print('val Loss: {:.4f}, Accuracy: {:.2f}%'
                                    .format(val_loss, val_acc))
        print('Completed in {:.0f}m {:.0f}s'
                                    .format(train_loss, train_acc))
    model.load_state_dict(best_model_wts)
    return model

In [19]:
base = train_baseline(model_base, train_loader,
                      val_loader, optimizer, EPOCH)

torch.save(base, 'baseline.pt')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


-----------------epoch 1 ----------------------
train Loss: 1.6396, Accuracy: 54.02%
val Loss: 1.6605, Accuracy: 53.21%
Completed in 2m 54s
-----------------epoch 2 ----------------------
train Loss: 0.9077, Accuracy: 73.07%
val Loss: 0.9490, Accuracy: 72.19%
Completed in 1m 73s
-----------------epoch 3 ----------------------
train Loss: 0.7060, Accuracy: 78.39%
val Loss: 0.7735, Accuracy: 76.67%
Completed in 1m 78s
-----------------epoch 4 ----------------------
train Loss: 0.5359, Accuracy: 83.40%
val Loss: 0.6166, Accuracy: 81.24%
Completed in 1m 83s
-----------------epoch 5 ----------------------
train Loss: 0.4456, Accuracy: 85.90%
val Loss: 0.5342, Accuracy: 84.09%
Completed in 0m 86s
-----------------epoch 6 ----------------------
train Loss: 0.3975, Accuracy: 87.85%
val Loss: 0.4963, Accuracy: 85.14%
Completed in 0m 88s
-----------------epoch 7 ----------------------
train Loss: 0.3427, Accuracy: 89.45%
val Loss: 0.4516, Accuracy: 86.23%
Completed in 0m 89s
-----------------epo

In [21]:
transform_base = transforms.Compose([transforms.Resize([64,64]),
                                     transforms.ToTensor()])
test_base = ImageFolder(root='/content/drive/MyDrive/Colab Notebooks/pytorch/작물 잎 사진/splitted/test',
                        transform=transform_base)
test_loader_base = torch.utils.data.DataLoader(test_base,
                                               batch_size = BATCH_SIZE, shuffle=True,
                                               num_workers=4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [23]:
baseline=torch.load('baseline.pt')
baseline.eval()
test_loss, test_accuracy = evaluate(baseline, test_loader_base)

print('baseline test acc: ', test_accuracy)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


baseline test acc:  93.87908374014269
